In [ ]:
import matplotlib.pyplot as plt           
import pandas as pd
from sklearn import  linear_model
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import re

Read words dictionary with probabilities

In [ ]:
words_dictionary = pd.read_csv('/content/drive/MyDrive/words_dictionary_probab.csv', encoding='latin-1')
vocabulary = set(words_dictionary['Unnamed: 0'])

words_dictionary.sort_values('Total', ascending = False)

,Unnamed: 0,Extremely Positive,Positive,Neutral,Negative,Extremely Negative,Total,1,2,3,4,5
23,covid,6284.0,11372.0,8301.0,9791.0,5430.0,41178.0,0.051608,0.061055,0.088592,0.061051,0.054252
232,price,1052.0,2295.0,1365.0,2747.0,1605.0,9064.0,0.008640,0.012322,0.014568,0.017129,0.016036
67,store,1487.0,2441.0,1592.0,1854.0,837.0,8211.0,0.012212,0.013106,0.016991,0.011560,0.008363
49,supermarket,1186.0,2127.0,1446.0,1958.0,1064.0,7781.0,0.009740,0.011420,0.015432,0.012209,0.010631
33,food,1042.0,1866.0,707.0,2079.0,1598.0,7292.0,0.008558,0.010018,0.007545,0.012963,0.015966
...,...,...,...,...,...,...,...,...,...,...,...,...
5412,glutenfre,3.0,1.0,0.0,1.0,1.0,6.0,0.000025,0.000005,0.000010,0.000006,0.000010
5417,luke,1.0,2.0,1.0,2.0,0.0,6.0,0.000008,0.000011,0.000011,0.000012,0.000009
5431,firstdayofspr,0.0,4.0,0.0,2.0,0.0,6.0,0.000008,0.000021,0.000010,0.000012,0.000009
5435,etsi,2.0,0.0,1.0,2.0,1.0,6.0,0.000016,0.000005,0.000011,0.000012,0.000010


Read data

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/test_token.csv', encoding='latin-1')
y_test = test["Sentiment"].map({"Extremely Positive":0, "Positive": 1,"Neutral": 2,"Negative": 3,"Extremely Negative": 4})
y_test = y_test.values
y_test = y_test[~test['TweetTokens'].isnull()]
test = test[~test['TweetTokens'].isnull()]
test

,Location,TweetAt,OriginalTweet,Sentiment,TweetTokens
0,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,"['trend', 'new', 'yorker', 'encount', 'empti',..."
1,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive,"['find', 'hand', 'sanit', 'fred', 'meyer', 'tu..."
2,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive,"['find', 'protect', 'love', 'one', 'covid']"
3,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative,"['panic', 'buy', 'hit', 'newyork', 'citi', 'an..."
4,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,"['toiletpap', 'dunnypap', 'covid', 'coronaviru..."
...,...,...,...,...,...
3790,Israel ??,16-03-2020,Meanwhile In A Supermarket in Israel -- People...,Positive,"['meanwhil', 'supermarket', 'israel', 'peopl',..."
3791,"Farmington, NM",16-03-2020,Did you panic buy a lot of non-perishable item...,Negative,"['panic', 'buy', 'lot', 'item', 'echo', 'need'..."
3792,"Haverford, PA",16-03-2020,Asst Prof of Economics @cconces was on @NBCPhi...,Neutral,"['asst', 'prof', 'econom', 'talk', 'recent', '..."
3793,NaN,16-03-2020,Gov need to do somethings instead of biar je r...,Extremely Negative,"['gov', 'need', 'someth', 'instead', 'biar', '..."


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/train_token.csv', encoding='latin-1')
y_train = train["Sentiment"].map({"Extremely Positive":0, "Positive": 1,"Neutral": 2,"Negative": 3,"Extremely Negative": 4})
y_train = y_train.values
y_train = y_train[~train['TweetTokens'].isnull()]
train = train[~train['TweetTokens'].isnull()]
train

,Location,TweetAt,OriginalTweet,Sentiment,TweetTokens
0,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,"['advic', 'talk', 'neighbour', 'famili', 'exch..."
1,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,"['covid', 'australia', 'woolworth', 'give', 'e..."
2,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive,"['food', 'stock', 'one', 'empti', 'pleas', 'pa..."
3,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,"['readi', 'go', 'supermarket', 'covid', 'outbr..."
4,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive,"['news', 'first', 'confirm', 'covid', 'case', ..."
...,...,...,...,...,...
41118,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral,"['airlin', 'pilot', 'offer', 'stock', 'superma..."
41119,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative,"['respons', 'complaint', 'provid', 'cite', 'co..."
41120,NaN,14-04-2020,You know itÂs getting tough when @KameronWild...,Positive,"['know', 'get', 'tough', 'ration', 'toilet', '..."
41121,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral,"['wrong', 'smell', 'hand', 'sanit', 'start', '..."


Calculate the apriror probabilities for classes

In [ ]:
N = np.zeros(5)
for i in range(0,5):
  N[i] = sum(y_train == i)
N = N/sum(N[:])
N

array([0.16109341, 0.27773049, 0.18672633, 0.24115372, 0.13329604])

## Make predictions

In [ ]:
def NaiveBayes(data,N,vocabulary):
  '''
  Input:
    data - dataframe of data
    N - probabilities for classes
    vocabulary - vocabulary of words from train data
  Output:
    y_pred - predictions
  '''

  y_pred = []

  for i in range(0,len(data)):
    pom = np.zeros(len(N)) 
    for word in re.sub("[^\w]", " ", data.iloc[i]['TweetTokens']).split():
      if word in vocabulary:
        pom = pom + np.log(np.array(words_dictionary[words_dictionary['Unnamed: 0']== word])[0][7:12].astype('float'))
    pom = pom + np.log(N)
    #pom = pom/sum(pom)
    y_pred.append(np.argmax(pom))

  return y_pred


In [ ]:
y_pred_test = NaiveBayes(test,N,vocabulary)
y_pred_train = NaiveBayes(train,N,vocabulary)

## Results

In [ ]:
m_train = confusion_matrix(y_train,y_pred_train)
print('Tacnost na trening podacima: ')
print(sum(m_train[i][i] for i in range(len(m_train))) / sum(sum(m_train))*100)

m = confusion_matrix(y_test,y_pred_test)
print('Tacnost na test podacima: ')
print(sum(m[i][i] for i in range(len(m))) / sum(sum(m))*100)
print('Konfuziona matrica na test podacima: ')
m

Tacnost na trening podacima: 
61.951895717308304
Tacnost na test podacima: 
47.82608695652174
Konfuziona matrica na test podacima: 


array([[315, 192,  25,  47,  20],
       [153, 395, 139, 187,  73],
       [ 18, 129, 336, 107,  26],
       [ 45, 195, 148, 445, 208],
       [  5,  38,  24, 201, 324]])

In [ ]:
#(m[0,0]+m[0,1]+m[1,0]+m[1,1]+m[2,2]+m[3,3]+m[3,4]+m[4,3]+m[4,4])/sum(sum(m)) * 100

In [ ]:
# Evaluating the model on the testing data set
#print(classification_report(y_test, y_pred))

In [ ]:
#missclassified = test[y_test!=y_pred]

Example of missclassified tweets

In [ ]:
#indeks = 5

#print(missclassified.iloc[indeks]['OriginalTweet'])
#print("")

#print(f"Sentiment {missclassified.iloc[indeks]['Sentiment']}")
#print(f"Prediction {y_pred[5]}")